# KMeans

Estaremos trabalhando com um conjunto de dados real sobre sementes, do repositório UCI: https://archive.ics.uci.edu/ml/datasets/seeds.

O grupo examinado foi formado por grãos pertencentes a três diferentes variedades de trigo: Kama, Rosa e Canadense, 70 elementos cada, selecionados aleatoriamente para
o experimento. A visualização de alta qualidade da estrutura interna do kernel foi detectada usando uma técnica de raio-X suave. É não destrutivo e consideravelmente mais barato do que outras técnicas de imagem mais sofisticadas, como microscopia de varredura ou tecnologia a laser. As imagens foram gravadas em placas de raios-X KODAK de 13x18 cm. Os estudos foram conduzidos usando grãos de trigo colhidos combinados originários de campos experimentais, explorados no Instituto de Agrofísica da Academia Polonesa de Ciências em Lublin.

O conjunto de dados pode ser usado para as tarefas de classificação e análise de cluster.


Informações sobre o atributo:

Para construir os dados, sete parâmetros geométricos de grãos de trigo foram medidos:
1. area A, 
2. perimeter P, 
3. compactness C = 4*pi*A/P^2, 
4. length of kernel, 
5. width of kernel, 
6. asymmetry coefficient 
7. length of kernel groove. 

Todos esses parâmetros eram contínuos com valor real.

Vamos ver se podemos agrupá-los em 3 grupos com K-médias!

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans

# Loads data.
dataset = spark.read.csv("seeds_dataset.csv",header=True,inferSchema=True)

In [3]:
dataset.head()

Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)

In [4]:
dataset.describe().show()

+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|summary|              area|         perimeter|         compactness|   length_of_kernel|   width_of_kernel|asymmetry_coefficient|   length_of_groove|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|  count|               210|               210|                 210|                210|               210|                  210|                210|
|   mean|14.847523809523816|14.559285714285718|  0.8709985714285714|  5.628533333333335| 3.258604761904762|   3.7001999999999997|  5.408071428571429|
| stddev|2.9096994306873647|1.3059587265640225|0.023629416583846364|0.44306347772644983|0.3777144449065867|   1.5035589702547392|0.49148049910240543|
|    min|             10.59|             12.41|              0.8081|              4.899|            

## Formate os dados

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [7]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [8]:
final_data = vec_assembler.transform(dataset)

## Dimensione os dados
É uma boa ideia dimensionar nossos dados para lidar com a maldição da dimensionalidade: https://en.wikipedia.org/wiki/Curse_of_dimensionality

In [9]:
from pyspark.ml.feature import StandardScaler

In [10]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [11]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [12]:
# Normalize cada recurso para ter um desvio padrão da unidade.
final_data = scalerModel.transform(final_data)

## Treine o modelo e avalie

In [13]:
# Trains a k-means model.
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [14]:
# Mostra o resultado.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 4.87257659 10.88120146 37.27692543 12.3410157   8.55443412  1.81649011
 10.32998598]
[ 4.06105916 10.13979506 35.80536984 11.82133095  7.50395937  3.27184732
 10.42126018]
[ 6.31670546 12.37109759 37.39491396 13.91155062  9.748067    2.39849968
 12.2661748 ]


In [15]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
|         2|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
+----------+
only showing top 20 rows

